In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Для сайта, продающего билеты в кино, имеется набор данных (логов) за период с сентября
по ноябрь.   
В примере показана таблица данных с дневной агрегацией. Она содержит:
1. Дату.
2. Идентификатор сессии пользователя.
3. Идентификатор пользователя.
4. Ярлык события
5. Доход от этих событий на определённую дату у определённого пользователя за одну
сессию.  

Используйте данные, чтобы рассчитать метрики отдельно за сентябрь, октябрь и ноябрь:
1. Количество сессий (визитов).
2. MAU.
3. DAU.
4. Revenue.
5. ARPU.
6. ARPPU.
7. Session / user.
8. Количество целевых действий (целевое действие — событие, за которое получен
доход).
Дополнительно рассчитайте показатель LTV на пользователя за период три месяца: с
сентября по ноябрь.

In [2]:
df = pd.read_excel('dataset-1.xlsx'
                   , names=['date', 'session', 'client', 'event', 'revenue']
                   , dtype={'date': str, 'session': str, 'client': int, 'event': str, 'revenue': float})
df.head()

,date,session,client,event,revenue
0,20180902.0,6517411217615201018blqo010gfv,42325189,Search Field,0.0
1,20180902.0,6517439917954201018jogp8guf,49083805,Movie cover,0.0
2,20180903.0,6517492978275201018xnvoyeqm,27598678,Event Place,0.0
3,20180903.0,651743624394010201018cg2zi5ol8z276c3x16y9zfr,5760746,Movie cover,0.0
4,20180903.0,6517492978275201018xnvoyeqm,27598678,Main - Menu,0.0


In [12]:
df.dtypes

date        object
session     object
client       int32
event       object
revenue    float64
dtype: object

In [3]:
df['date'] = pd.to_datetime(df.date, format='%Y%m%d.0')
df.head()

,date,session,client,event,revenue
0,2018-09-02,6517411217615201018blqo010gfv,42325189,Search Field,0.0
1,2018-09-02,6517439917954201018jogp8guf,49083805,Movie cover,0.0
2,2018-09-03,6517492978275201018xnvoyeqm,27598678,Event Place,0.0
3,2018-09-03,651743624394010201018cg2zi5ol8z276c3x16y9zfr,5760746,Movie cover,0.0
4,2018-09-03,6517492978275201018xnvoyeqm,27598678,Main - Menu,0.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20342 entries, 0 to 20341
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     20342 non-null  datetime64[ns]
 1   session  20342 non-null  object        
 2   client   20342 non-null  int32         
 3   event    20342 non-null  object        
 4   revenue  20342 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 715.3+ KB


1. Количество сессий (визитов).


In [8]:
temp = df.groupby(df.date.dt.month_name(), sort=False) \
    .agg(session_cnt=('session', 'nunique'))
temp.loc[''] = temp.session_cnt.sum()
temp

,session_cnt
date,
September,2827
October,9995
November,4086
,16908


2. MAU.


In [13]:
(df.groupby(df.date.dt.month_name(), sort=False) 
    .agg(
        session_cnt=('session', 'nunique')
        , mau=('client', 'nunique'))
)

,session_cnt,mau
date,,
September,2827,2511
October,9995,8520
November,4086,3647


3. DAU.


In [10]:
temp = df.groupby([df.date.dt.month_name(), df.date.dt.day], sort=False).agg(mau=('client', 'nunique'))


In [12]:
temp.index = temp.index.set_names(['month', 'day'])
temp.head()

mau
month     day     
September 2      2
          3      2
          5      1
          6      2
          7      1

4. Revenue.


In [14]:
(df.groupby(df.date.dt.month_name(), sort=False) 
    .agg(
        session_cnt=('session', 'nunique')
        , mau=('client', 'nunique')
        , revenue=('revenue', 'sum'))
)


,session_cnt,mau,revenue
date,,,
September,2827,2511,2.779013e+07
October,9995,8520,2.986842e+07
November,4086,3647,1.960508e+07


5. ARPU.


In [15]:
(df.groupby(df.date.dt.month_name(), sort=False) 
    .agg(
        session_cnt=('session', 'nunique')
        , mau=('client', 'nunique')
        , revenue=('revenue', 'sum'))
    .assign(arpu=lambda x: x.revenue / x.mau)
)

,session_cnt,mau,revenue,arpu
date,,,,
September,2827,2511,2.779013e+07,11067.353869
October,9995,8520,2.986842e+07,3505.682983
November,4086,3647,1.960508e+07,5375.674394


6. ARPPU.


In [17]:
df['paying_client'] = df.apply(lambda x: f'{x.client}' if x.revenue else np.nan, axis=1)
df.iloc[[0, 23]]

,date,session,client,event,revenue,paying_client
0,2018-09-02,6517411217615201018blqo010gfv,42325189,Search Field,0.000000,NaN
23,2018-09-12,651829401017921201018ym2qrmpl,2888955,Place Select,18816.599804,2888955


In [18]:
(df.groupby(df.date.dt.month_name(), sort=False) 
    .agg(
        session_cnt=('session', 'nunique')
        , mau=('client', 'nunique')
        , mapu=('paying_client', 'nunique')
        , revenue=('revenue', 'sum')
        )
    .assign(arpu=lambda x: round(x.revenue / x.mau, 2))
    .assign(arppu=lambda x: round(x.arpu * (x.mau / x.mapu), 2))
)

,session_cnt,mau,mapu,revenue,arpu,arppu
date,,,,,,
September,2827,2511,1268,2.779013e+07,11067.35,21916.50
October,9995,8520,1195,2.986842e+07,3505.68,24994.47
November,4086,3647,890,1.960508e+07,5375.67,22028.17


7. Session / user.


In [37]:
res = (df.groupby(df.date.dt.month_name(), sort=False) 
    .agg(
        session_cnt=('session', 'nunique')
        , mau=('client', 'nunique')
        , mapu=('paying_client', 'nunique')
        , revenue=('revenue', 'sum')
        )
    .assign(arpu=lambda x: round(x.revenue / x.mau, 2))
    .assign(arppu=lambda x: round(x.arpu * (x.mau / x.mapu), 2))
    .assign(session_per_client=lambda x: round(x.session_cnt / x.mau, 2))
)
res

,session_cnt,mau,mapu,revenue,arpu,arppu,session_per_client
date,,,,,,,
September,2827,2511,1268,2.779013e+07,11067.35,21916.50,1.13
October,9995,8520,1195,2.986842e+07,3505.68,24994.47,1.17
November,4086,3647,890,1.960508e+07,5375.67,22028.17,1.12


8. Количество целевых действий (целевое действие — событие, за которое получен
доход).


действия фиксируются в столбце ярлык

In [35]:
(df[df.revenue > 0].groupby([df.date.dt.month_name(), 'event'], sort=False)
    .agg(cnt=('client', 'count'))
)    

cnt
date      event                         
September Place Select               550
          Promocode                   68
          Desktop - To Payment Card  421
          Mobile - To Payment Card   580
          Search Field                 1
October   Place Select               413
          Mobile - To Payment Card   601
          Promocode                   46
          Desktop - To Payment Card  457
          Movie cover                  1
November  Mobile - To Payment Card   471
          Promocode                   30
          Desktop - To Payment Card  317
          Place Select               314
          Movie cover                  1

In [36]:
(df[df.revenue > 0].groupby('event', sort=False)
    .agg(cnt=('client', 'count'))
)    

,cnt
event,
Place Select,1277
Promocode,144
Desktop - To Payment Card,1195
Mobile - To Payment Card,1652
Search Field,1
Movie cover,2


Дополнительно рассчитайте показатель LTV на пользователя за период три месяца: с
сентября по ноябрь.

In [108]:
lt = (df.groupby(['client', df.date.dt.month_name(),])
    .agg(temp=('session', 'count'))
    # .apply(lambda x: pd.Series(1, index=['cnt']), axis=1)
    .reset_index(1)
    .drop('temp', axis=1)
    .groupby('client')
    .agg(cnt=('date', 'count'))
    .mean()
)
lt

cnt    1.065708
dtype: float64

In [38]:
arpu = res.arpu.mean()
arpu

6649.566666666667

In [113]:
ltv = arpu * lt
ltv

cnt    7086.498187
dtype: float64

In [70]:
df.head()

,date,session,client,event,revenue
0,2018-09-02,6517411217615201018blqo010gfv,42325189,Search Field,0.0
1,2018-09-02,6517439917954201018jogp8guf,49083805,Movie cover,0.0
2,2018-09-03,6517492978275201018xnvoyeqm,27598678,Event Place,0.0
3,2018-09-03,651743624394010201018cg2zi5ol8z276c3x16y9zfr,5760746,Movie cover,0.0
4,2018-09-03,6517492978275201018xnvoyeqm,27598678,Main - Menu,0.0
